In [ ]:
#import libraries
import requests
import pandas as pd
import time

In [ ]:
#Keys
API_KEY = "xxxx"
CHANNEL_ID = "xxxx" #my channel ID
CHANNEL_ID2 = "xxxx"  # StrataScratch channel ID

In [ ]:
#Example of working with github api
response = requests.get('https://api.github.com').json()

In [ ]:
response

{'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'current_user_repositories_url': 'https://api.github.com/user/repos{?type,page,per_page,sort}',
 'current_user_url': 'https://api.github.com/user',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={q

In [ ]:
#Working with Youtube API and my channel ID to retrieve uploaded video details
pageToken = ""
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
response = requests.get(url).json()
response

{'etag': 'UBAOiXjvY0CZ7Ub9tJDO9bUtsU8',
 'items': [],
 'kind': 'youtube#searchListResponse',
 'pageInfo': {'resultsPerPage': 0, 'totalResults': 0},
 'regionCode': 'US'}

Since I didn't upload any videos, I got empty results

In [ ]:
#Working with Youtube API and StrataScratch channel ID
pageToken = ""
#Below line is written after referring https://developers.google.com/youtube/v3/docs/search/list page
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID2+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
response = requests.get(url).json()
response

{'etag': 'OY5YlYcXsniAbtk5QrjsyD43i7I',
 'items': [{'etag': 's1tr7m1aFScZW8QZ2gbyRJTfoWs',
   'id': {'kind': 'youtube#video', 'videoId': 'jN5hpgBrz8k'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
    'channelTitle': 'StrataScratch',
    'description': 'This is a really interesting Python question that is frequently asked at Meta/Facebook data science interviews. The title of this ...',
    'liveBroadcastContent': 'none',
    'publishTime': '2022-03-22T17:30:06Z',
    'publishedAt': '2022-03-22T17:30:06Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/jN5hpgBrz8k/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/jN5hpgBrz8k/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/jN5hpgBrz8k/mqdefault.jpg',
      'width': 320}},
    'title': 'Dealing With Python Date Manipulations In a Meta/Facebook Data Sc

In [ ]:
response['items'][0]['snippet']['description']

'This is a really interesting Python question that is frequently asked at Meta/Facebook data science interviews. The title of this ...'

# Getting all StrataScratch Videos details

In [ ]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    try:
      dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    except:
      dislike_count = response_video_stats.get('dislikeCount', 0)
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, dislike_count, comment_count

In [ ]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID2+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                #print(video_id)
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"dislike_count":dislike_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [ ]:
YouTubeDF = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"])
YouTubeDF = get_videos(YouTubeDF)

In [ ]:
YouTubeDF

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,jN5hpgBrz8k,Dealing With Python Date Manipulations In a Me...,2022-03-22,3247,105,0,20
1,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,2022-03-10,2025,62,0,2
2,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2022-03-02,2620,76,0,9
3,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08,5707,179,0,20
4,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27,3252,90,0,6
...,...,...,...,...,...,...,...
80,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,1995,12,0,2
81,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1313,22,0,0
82,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1030,21,0,0
83,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,7766,70,0,1


Now let's store the data in a CSV file

In [ ]:
YouTubeDF.to_csv('youtube_vids_info_pull.csv')

In [ ]:
from google.colab import files
files.download('youtube_vids_info_pull.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>